        OpenAI Agents SDK

            Handoffs

Handoffs allow an agent to delegate tasks to another agent. This is particularly useful in scenarios where different agents specialize in distinct areas. For example, a customer support app might have agents that each specifically handle tasks like order status, refunds, FAQs, etc.

Handoffs are represented as tools to the LLM. So if there's a handoff to an agent named Refund Agent, the tool would be called transfer_to_refund_agent.

Creating A Handoff

All agents have a handoffs param, which can either take an Agent directly, or a Handoff object that customizes the Handoff.

You can create a handoff using the handoff() function provided by the Agents SDK. This function allows you to specify the agent to hand off to, along with optional overrides and input filters.

Basic Usage

Here's how you can create a simple handoff:


ہینڈ آف ایک ایجنٹ کو کام دوسرے ایجنٹ کو سونپنے کی اجازت دیتا ہے۔ یہ خاص طور پر ایسے حالات میں مفید ہے جہاں مختلف ایجنٹ الگ الگ علاقوں میں مہارت رکھتے ہیں۔ مثال کے طور پر، ایک کسٹمر سپورٹ ایپ میں ایسے ایجنٹ ہو سکتے ہیں جو ہر ایک خاص طور پر آرڈر کی حیثیت، ریفنڈز، عمومی سوالنامہ وغیرہ جیسے کاموں کو ہینڈل کرتے ہیں۔

ہینڈ آف کو LLM کے ٹولز کے طور پر پیش کیا جاتا ہے۔ لہذا اگر ریفنڈ ایجنٹ نامی ایجنٹ کو کوئی ہینڈ آف ہے تو اس ٹول کو ٹرانسفر_ٹو_ریفنڈ_ایجنٹ کہا جائے گا۔

ایک ہینڈ آف بنانا

تمام ایجنٹوں کے پاس ہینڈ آف پرم ہوتا ہے، جو یا تو براہ راست ایجنٹ لے سکتا ہے، یا ہینڈ آف آبجیکٹ جو ہینڈ آف کو اپنی مرضی کے مطابق بناتا ہے۔

آپ ایجنٹس SDK کے ذریعہ فراہم کردہ ہینڈ آف() فنکشن کا استعمال کرتے ہوئے ایک ہینڈ آف بنا سکتے ہیں۔ یہ فنکشن آپ کو اختیاری اوور رائیڈز اور ان پٹ فلٹرز کے ساتھ ایجنٹ کے حوالے کرنے کی وضاحت کرنے کی اجازت دیتا ہے۔

بنیادی استعمال

یہاں یہ ہے کہ آپ ایک سادہ ہینڈ آف کیسے بنا سکتے ہیں:

In [ ]:
from agents import Agent, handoff

billing_agent = Agent(name="Billing agent")
refund_agent = Agent(name="Refund agent")


triage_agent = Agent(name="Triage agent", handoffs=[billing_agent, handoff(refund_agent)])

Customizing handoffs via the handoff() function

The handoff() function lets you customize things.

* agent: This is the agent to which things will be handed off.
* tool_name_override: By default, the Handoff.default_tool_name() function is used, which resolves to
  transfer_to_<agent_name>. You can override this.
* tool_description_override: Override the default tool description from Handoff.default_tool_description
  ()
* on_handoff: A callback function executed when the handoff is invoked. This is useful for things like 
  kicking off some data fetching as soon as you know a handoff is being invoked. This function receives the agent context, and can optionally also receive LLM generated input. The input data is controlled by the input_type param.
* input_type: The type of input expected by the handoff (optional).
* input_filter: This lets you filter the input received by the next agent. See below for more.
  is_enabled: Whether the handoff is enabled. This can be a boolean or a function that returns a boolean, allowing you to dynamically enable or disable the handoff at runtime.

In [ ]:
from agents import Agent, handoff, RunContextWrapper

def on_handoff(ctx: RunContextWrapper[None]):
    print("Handoff called")

agent = Agent(name="My agent")

handoff_obj = handoff(
    agent=agent,
    on_handoff=on_handoff,
    tool_name_override="custom_handoff_tool",
    tool_description_override="Custom description",
)

Handoff inputs

In certain situations, you want the LLM to provide some data when it calls a handoff. For example, imagine a handoff to an "Escalation agent". You might want a reason to be provided, so you can log it.

ہینڈ آف ان پٹس

بعض حالات میں، آپ چاہتے ہیں کہ LLM کچھ ڈیٹا فراہم کرے جب وہ ہینڈ آف کو کال کرے۔ مثال کے طور پر، ایک "Escalation agent" کے حوالے کرنے کا تصور کریں۔ ہو سکتا ہے کہ آپ کو کوئی وجہ فراہم کی جائے، تاکہ آپ اسے لاگ ان کر سکیں۔

In [ ]:
from pydantic import BaseModel

from agents import Agent, handoff, RunContextWrapper

class EscalationData(BaseModel):
    reason: str

async def on_handoff(ctx: RunContextWrapper[None], input_data: EscalationData):
    print(f"Escalation agent called with reason: {input_data.reason}")

agent = Agent(name="Escalation agent")

handoff_obj = handoff(
    agent=agent,
    on_handoff=on_handoff,
    input_type=EscalationData,
)

Input filters

When a handoff occurs, it's as though the new agent takes over the conversation, and gets to see the entire previous conversation history. If you want to change this, you can set an input_filter. An input filter is a function that receives the existing input via a HandoffInputData, and must return a new HandoffInputData.

There are some common patterns (for example removing all tool calls from the history), which are implemented for you in agents.extensions.handoff_filters

ان پٹ فلٹرز

جب کوئی ہینڈ آف ہوتا ہے، تو ایسا لگتا ہے جیسے نیا ایجنٹ بات چیت کو سنبھالتا ہے، اور پوری پچھلی گفتگو کی تاریخ دیکھتا ہے۔ اگر آپ اسے تبدیل کرنا چاہتے ہیں، تو آپ ایک input_filter سیٹ کر سکتے ہیں۔ ایک ان پٹ فلٹر ایک فنکشن ہے جو ایک HandoffInputData کے ذریعے موجودہ ان پٹ وصول کرتا ہے، اور اسے ایک نیا HandoffInputData واپس کرنا ضروری ہے۔

کچھ عام پیٹرن ہیں (مثال کے طور پر تمام ٹول کالز کو ہسٹری سے ہٹانا)، جو آپ کے لیے agents.extensions.handoff_filters میں لاگو کیے گئے ہیں۔

In [ ]:
from agents import Agent, handoff
from agents.extensions import handoff_filters

agent = Agent(name="FAQ agent")

handoff_obj = handoff(
    agent=agent,
    input_filter=handoff_filters.remove_all_tools, 
)

Recommended prompts

To make sure that LLMs understand handoffs properly, we recommend including information about handoffs in your agents. We have a suggested prefix in agents.extensions.handoff_prompt.RECOMMENDED_PROMPT_PREFIX, or you can call agents.extensions.handoff_prompt.prompt_with_handoff_instructions to automatically add recommended data to your prompts.

تجویز کردہ اشارے

اس بات کو یقینی بنانے کے لیے کہ LLMs ہینڈ آف کو صحیح طریقے سے سمجھتے ہیں، ہم آپ کے ایجنٹوں میں ہینڈ آف کے بارے میں معلومات شامل کرنے کی تجویز کرتے ہیں۔ ہمارے پاس agents.extensions.handoff_prompt.RECOMMENDED_PROMPT_PREFIX میں ایک تجویز کردہ سابقہ ​​ہے، یا آپ agents.extensions.handoff_prompt.prompt_with_handoff_instructions کو کال کر سکتے ہیں تاکہ اپنے پرامپٹس میں تجویز کردہ ڈیٹا کو خودکار طور پر شامل کریں۔

In [ ]:
from agents import Agent
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

billing_agent = Agent(
    name="Billing agent",
    instructions=f"""{RECOMMENDED_PROMPT_PREFIX}
    <Fill in the rest of your prompt here>.""",
)